<a href="https://colab.research.google.com/github/RedPandaCreations/AlteraAudio/blob/main/AlteraAudio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Altera Audio
By `Ethan Marks`

## About

AlteraAudio is a program written in the Python programming language designed to seamlessly download YouTube videos and audio. It simplifies the download process, eliminating the need for complex setups or external tools. In addition to offering a user-friendly interface with non-programmer users in mind, AlteraAudio also has a key advantage over other downloaders: the ability to preserve metadata. If the user elects to download a song, this functionality ensures that downloaded audio files retain crucial information such as artist name, song title, and album name. AlteraAudio is built on Google Colab, a cloud-based computing platform that allows you to run code in your browser without downloading or installing anything. Now that you're familiar with AlteraAudio's capabilities, let's delve into the setup process in the next cell.

# Setup

These cells will download and install the requisite packages, as well as define all functions needed to run everything.

The notebook should run them automatically, but you can run them manually by pressing `Ctrl+F9` to run every cell in the notebook.

It usually takes about 40 seconds to run, although that's just an estimate.

You're more than welcome to read through each cell, although you don't need to understand the code to use it.

Once you've run the cells, go to the [interface](#scrollTo=fbosCE6PuvNS).

In [ ]:
# @title Install Packages
!pip install pytube
!pip install mutagen
!pip install yt-dlp

In [ ]:
# @title Import Packages
import os

# Pytube is used to get the metadata and playlist structures
from pytube import YouTube, Playlist
# yt-dlp is used to download the videos
import yt_dlp

# Requests is used to download the album art from YouTube's database, as Pytube only outputs the url, not the actual image data.
import requests

# Mutagen is used to modify audio metadata.
import mutagen

# Zipfile is used to compress many files into one big file for ease of downloading.
import zipfile

# This module allows AlteraAudio to download the files directly to the user device, as otherwise the user would have to root through the environment files to get their video.
# We want the script to be runable in any environment with minimal code modifications, so the logic around the import essentially checks if it's being run in Colab.
try:
  # We import colab's `files` module to facilitate downloads
  from google.colab import files
  # ipywidgets are used to create the progress bar
  from IPython.display import display
  from ipywidgets import FloatProgress, IntProgress
  # ipywidgets are also used for the user interface
  from ipywidgets import Dropdown, Checkbox, Text, Label, Layout, VBox, Button
  environment = "colab"
except ImportError:
  environment = "other"

In [ ]:
# @title Fix Pytube

# YouTube likes to change things to combat adblockers, and PyTube takes a while releasing updates to fix it.
# This means that until PyTube releases an update, AlteraAudio has to make various changes to PyTube's code to circumvent YouTube's adblocker-blocker

from pytube.innertube import _default_clients
from pytube import cipher
import re

_default_clients["ANDROID"]["context"]["client"]["clientVersion"] = "19.08.35"
_default_clients["IOS"]["context"]["client"]["clientVersion"] = "19.08.35"
_default_clients["ANDROID_EMBED"]["context"]["client"]["clientVersion"] = "19.08.35"
_default_clients["IOS_EMBED"]["context"]["client"]["clientVersion"] = "19.08.35"
_default_clients["IOS_MUSIC"]["context"]["client"]["clientVersion"] = "6.41"
_default_clients["ANDROID_MUSIC"] = _default_clients["ANDROID_CREATOR"]

def get_throttling_function_name(js: str) -> str:
    """Extract the name of the function that computes the throttling parameter.

    :param str js:
        The contents of the base.js asset file.
    :rtype: str
    :returns:
        The name of the function used to compute the throttling parameter.
    """
    function_patterns = [
        r'a\.[a-zA-Z]\s*&&\s*\([a-z]\s*=\s*a\.get\("n"\)\)\s*&&\s*'
        r'\([a-z]\s*=\s*([a-zA-Z0-9$]+)(\[\d+\])?\([a-z]\)',
        r'\([a-z]\s*=\s*([a-zA-Z0-9$]+)(\[\d+\])\([a-z]\)',
    ]
    #logger.debug('Finding throttling function name')
    for pattern in function_patterns:
        regex = re.compile(pattern)
        function_match = regex.search(js)
        if function_match:
            #logger.debug("finished regex search, matched: %s", pattern)
            if len(function_match.groups()) == 1:
                return function_match.group(1)
            idx = function_match.group(2)
            if idx:
                idx = idx.strip("[]")
                array = re.search(
                    r'var {nfunc}\s*=\s*(\[.+?\]);'.format(
                        nfunc=re.escape(function_match.group(1))),
                    js
                )
                if array:
                    array = array.group(1).strip("[]").split(",")
                    array = [x.strip() for x in array]
                    return array[int(idx)]

    raise RegexMatchError(
        caller="get_throttling_function_name", pattern="multiple"
    )

cipher.get_throttling_function_name = get_throttling_function_name

In [ ]:
# @title Define Auxilary Functions
def log_message(message):
    # This function is seperate for the convenience of any developers looking to use AlteraAudio's code in their own projects.
    # It allows them to add extra lines of code to this function, for example to write the output to a .txt file for recordkeeping.
    print(message)

def sanitize_to_ascii(input_string):
    # Encode the string to ASCII and ignore characters that cannot be encoded
    ascii_string = input_string.encode('ascii', 'ignore').decode('ascii')
    return ascii_string

def is_playlist_or_video(url):
    # Since the user can input the urls of individual videos or entire playlists, This function allows AlteraAudio to distinguish between them.
    if 'playlist?list=' in url:
        return "Playlist"
    try:
        yt = YouTube(url)
        if yt.video_id:
            return "Video"
        else:
            raise Exception (f"Invalid url")
    except Exception as e:
        raise Exception (f"Encountered an error: {e}")

def strip_filename(filename):
    # AlteraAudio needs to work on Windows environments as well as Linux ones, and Windows imposes limitations on filenames.
    # Since the filename is determined by the video title, and YouTube has much laxer naming restrictions,
    # this function sanitizes the filenames so that if a video title contains disallowed characters, it doesn't break the system.
    disallowed_chars = ['<', '>', ':', '"', '/', '\\', '|', '?', '*']
    for char in disallowed_chars:
        filename = filename.replace(char, '')
    return filename

def process_name(albumname):
    # This function removes some of Youtube/Pytube's formatting nonsense to do with playlist names.
    album_removed = albumname.replace('Album - ', '')
    topic_removed = album_removed.replace(' - Topic', '')
    return(topic_removed)

def compress_to_zip(zip_file_path, file_paths):
    # This function compresses a folder full of files into a single zip file.
    with zipfile.ZipFile(zip_file_path, 'w') as zipf:
        for file_path in file_paths:
            zipf.write(file_path, os.path.basename(file_path))
        for file_path in file_paths:
            try:
              os.remove(file_path)
            except:
              # If we raise an error here, the user has to restart everything. Instead we just print a warning message
              # Since removing files after we've compressed them is a superfluous convenience, it's fine that we skip it.
              print("WARNING: Failed to remove file, continuing with others.")

def url_sanitize(input_string):
    # Remove the unnecessary sections of the url that would cause problems downstream
    index = input_string.find("&list=")
    if index == -1:
        return input_string
    important_information = input_string[:index]
    return important_information

In [ ]:
# @title Define Core Functions
def get_metadata(url="https://www.youtube.com/watch?v=dQw4w9WgXcQ"):
    """
    Uses pytube to get information about a specified url.

    Returns a list of the form:

    [title, channel, year, cover]

    Example:

    ['Never Gonna Give You Up', 'Rick Astley', 2009, 'https://i.ytimg.com/vi/dQw4w9WgXcQ/hq720.jpg?sqp=-oaymwEXCNUGEOADIAQqCwjVARCqCBh4INgESFo&rs=AOn4CLBX-HcaMSEAucUr5J0qD5nEyiPAoQ']
    """
    yt = YouTube(url)

    title = sanitize_to_ascii(yt.title)
    channel = process_name(yt.author)
    date = yt.publish_date
    year = date.year
    cover = yt.thumbnail_url

    metadata_list = [title, channel, year, cover]

    return(metadata_list)

def attach_metadata(path = '', metadata = [], genre='', albArtist=''):
    """
    Uses Mutagen to attach metadata to the audio file.

    Metadata format: [title, channel, year, cover, album, track]
    """
    try:
      # Try to load the audio file with ID3 tags
      audio = mutagen.id3.ID3(path)
    except mutagen.id3.ID3NoHeaderError:
      # If there's no ID3 header (no tags), create a new empty ID3 object
      audio = mutagen.id3.ID3()

    audio["TIT2"] = mutagen.id3.TIT2(encoding=3, text=metadata[0])
    audio["TPE1"] = mutagen.id3.TPE1(encoding=3, text=metadata[1])
    audio["TDRC"] = mutagen.id3.TDRC(encoding=3, text=str(metadata[2]))
    audio["TALB"] = mutagen.id3.TALB(encoding=3, text=metadata[4])
    # These two are different because they were implemented at a different date than the rest,
    if genre != '':
      audio["TCON"] = mutagen.id3.TCON(encoding=3, text=genre)
    if albArtist != '':
      audio["TPE2"] = mutagen.id3.TPE2(encoding=3, text=albArtist)
    # Although Mutagen is capable of it, we don't use the genre tag because Youtube doesn't categorize videos with genres (or at least, pytube doesn't access it)

    if len(metadata) >= 6:
        audio["TRCK"] = mutagen.id3.TRCK(encoding=3, text=str(metadata[5]))

    audio.save()

    # Metadata[3] is out of sequence from the rest of the tags because it corrosponds to the cover art, which is a more involved process
    change_cover_art(path, metadata[3])

def change_cover_art(path, image):
    # Download the image from the URL
    image_response = requests.get(image)
    image_data = image_response.content

    audio = mutagen.id3.ID3(path)

    # Remove existing cover art (if any)
    audio.delall('APIC')

    # These are the cover parameters
    mime_type = 'image/jpeg'
    image_type = 3
    description = "thumbnail"

    audio.add(mutagen.id3.APIC(
        encoding=3,  # utf-8
        mime=mime_type,
        type=image_type,
        desc=description,
        data=image_data
    ))

    # Save the audio file once it's been modified
    audio.save()

def download_video(url="https://www.youtube.com/watch?v=dQw4w9WgXcQ", metadata=[], format='m4a', directory=''):
    """
    Downloads YouTube videos or audio to a specified directory using yt-dlp.

    Inputs:
    - url: The video URL. Cannot be a playlist; playlist processing must be handled elsewhere.
    - metadata: Metadata for logging and formatting, structured as [title, channel, year, cover, album, track].
    - format: Must be either 'mp3', 'm4a, or 'mp4'. Determines the type of download.
    - directory: The output directory where files will be saved.

    Notes:
    - yt-dlp provides better support for handling various formats compared to pytube.
    """
    try:
        title = metadata[0] if metadata else YouTube(url).title

        # Construct the file path
        stripped_name = strip_filename(title)
        file_path = os.path.join(directory, f"{stripped_name}.{format}")

        # Prepare output format
        if format == 'mp3':
            output_format = 'bestaudio[ext=mp3]/best'
            extension = 'mp3'
        elif format == 'm4a':
            output_format = 'bestaudio[ext=m4a]'
            extension = 'm4a'
        elif format == 'mp4':
            output_format = 'bestvideo[ext=mp4][height<=1080]+bestaudio[ext=m4a]'
            extension = 'mp4'
        else:
            raise ValueError(f"Invalid format: '{format}'")

        # Setup yt-dlp options
        ydl_opts = {
            'format': output_format,
            'outtmpl': os.path.join(directory, stripped_name),
            'postprocessors': [{
                'key': 'FFmpegExtractAudio',
                'preferredcodec': format,
                'preferredquality': '192'
            }] if (format == 'mp3' or format == 'm4a') else None,
            'merge_output_format': extension if format == 'mp4' else None,
            'quiet': True
        }

        # Download the video/audio
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(url, download=True)

        return file_path

    except Exception as e:
        raise Exception(f"Encountered an error: {e}")

def altera(input="https://www.youtube.com/watch?v=dQw4w9WgXcQ", format='m4a', output='output', genre='', albArtist='', sequential=True, apply_metadata=True):
    """
    The main function for AlteraAudio. Takes a youtube link and saves a video to the output.

    Examples:
        altera('https://www.youtube.com/watch?v=dQw4w9WgXcQ', 'mp4')
            outputs: saves a video of 'Never Gonna Give You Up' to the output
    """

    # If the input is example, set url to the example video; otherwise sanitize and set url to the input
    if input == "example":
        url = "https://www.youtube.com/watch?v=dQw4w9WgXcQ"
    else:
        url = url_sanitize(input)

    # This section gets the video URLs
    if is_playlist_or_video(url) == "Video":
        # For execution flow simplicity, we still use a list even for a single video
        videos = [url]

        # If we're downloading a single song, we use the title of the video as the album name
        album = YouTube(url).title

        # Also, we aren't compressing the playlist so we don't need the extra step.
        progress_max_addition = 0
    else:
        # PyTube has inbuilt functionality for getting a list of URLs from a playlist, saving us from making a for loop
        playlist = Playlist(url)
        videos = playlist.video_urls

        # We use the name of the playlist (minus the formatting that we remove with the process_name function) as the album name
        album = process_name(playlist.title)

        # Because we're also compressing the playlist, we need an extra step in the progress bar
        progress_max_addition = 1

    # This section determines how many steps are in each iteration for the progress bar
    # The full step list is as follows: downloading, metadata
    if format == 'mp4':
        # If we're just downloading the video and nothing else, there is one step
        progress_updates = 1
    elif format == 'mp3' or format == 'm4a':
        # Otherwise there are the full two steps: downloading and metadata
        progress_updates = 2

    # This is the list of finished file paths. For execution flow simplicity, we still use a list even for a single video
    file_paths = []

    # Log a few of the parameters
    log_message(f"Downloading '{album}' as .{format} file(s)")

    # Instantiate the progress bar
    progress_bar = FloatProgress(min=0, max=len(videos) + progress_max_addition, description="0/" + str(len(videos)))
    progress_bar.style = {'bar_style': 'info'}
    display(progress_bar)

    # Main processing loop.
    for i in range(len(videos)):
        # Get the video URL we're using this iteration
        video_url = videos[i]

        # This is the metadata section, which we only call if we're downloading an audio file with the metadata option enabled
        if apply_metadata and (format == 'mp3' or format == 'm4a'):
            metadata = get_metadata(video_url)

            # We append the album name separately, because it depends on whether we're downloading a single song or an entire playlist, which the metadata() function doesn't have access to
            metadata.append(album)

            # Sequential provides an option for the edge case where the user is downloading a playlist as audio files, but they aren't in any particular order
            if sequential:
                metadata.append(i + 1)
        else:
            # We set metadata as an empty list if we don't need it
            metadata = []

        # Download the video
        downloaded_path = download_video(video_url, metadata, format, output)

        # Update the progress bar now we've downloaded the video
        progress_bar.value += 1 / progress_updates

        # Attach metadata if required
        if (format == 'mp3' or format == 'm4a') and apply_metadata and metadata:
            # Add Metadata to the audio file
            attach_metadata(downloaded_path, metadata, genre=genre, albArtist=albArtist)

            # Update the progress bar now we've updated the metadata
            progress_bar.value += 1 / progress_updates

        # Update the progress bar description
        progress_bar.description = (str(i + 1) + "/" + str(len(videos)))

        # Check for duplicates
        if downloaded_path in file_paths:
            print(f"Warning: Downloaded path '{downloaded_path}' already exists in file_paths list.")
        else:
            file_paths.append(downloaded_path)

    if len(videos) == 1:
        # If we're only downloading one file (or if it's a playlist containing only one video), there's no need to zip up the files, so we just return the single file.

        # Mark the progress bar as complete
        progress_bar.bar_style = 'success'
        return file_paths[0]

    # Compress every file we just downloaded into a zip file
    progress_bar.description = "Compressing"
    zip_path = os.path.join(output, album + '.zip')

    compress_to_zip(zip_path, file_paths)

    # Mark the progress bar as complete once we've compressed the files
    progress_bar.value += 1
    progress_bar.bar_style = 'success'
    progress_bar.description = "Done!"
    return zip_path

# Terms of Service

## Disclaimer

I, the developer of AlteraAudio, am not a lawyer, and this document is not legal advice. The following Terms of Service (TOS) for AlteraAudio are provided for informational purposes only. It is your responsibility to ensure compliance with YouTube's Terms of Service and applicable laws.

## Acceptance of Terms

By using AlteraAudio, you agree to abide by these terms and conditions. If you do not agree with any part of these terms, refrain from using AlteraAudio.

## Personal Use Only

AlteraAudio is intended solely for personal use. It is your responsibility to ensure that your use of AlteraAudio complies with all applicable laws and regulations. Any unauthorized use, including but not limited to commercial use, distribution, or reproduction of downloaded content, may violate YouTube's Terms of Service and could result in legal consequences.

## Compliance with YouTube's Terms of Service

AlteraAudio is not endorsed, affiliated, or approved by YouTube. The use of AlteraAudio to download content from YouTube may be against YouTube's Terms of Service. It is your responsibility to review and comply with YouTube's Terms of Service, which can be found at [YouTube Terms of Service](https://www.youtube.com/t/terms).

## No Warranty

AlteraAudio is provided "as is" without any warranty, express or implied. I make no representations or warranties regarding the functionality, reliability, or security of the software.

## Limitation of Liability

I will not be liable for any indirect, incidental, special, consequential, or punitive damages, or any loss of profits or revenues, whether incurred directly or indirectly.

## Changes to the Terms

I reserve the right to modify these terms at any time without notice. It is your responsibility to review these terms regularly to stay informed of any changes.

## Apache 2.0 License

In addition to the terms above, AlteraAudio is licensed under the [Apache License 2.0](https://www.apache.org/licenses/LICENSE-2.0.html). This means you can freely use, modify, and distribute the software, as long as you:

 - Include the copyright notice and license with the software.
 - Give credit to the original author (me).

## Contact

For any questions, suggestions, bug reports, or other subjects that require my attention, please email me at [29sr494d5@mozmail.com](mailto:29sr494d5@mozmail.com?subject=AlteraAudio%20Support)

# Usage

## Usage Guide

### Opening the Interface

In order to use AlteraAudio, you need to configure your settings (such as the video url and whether you want to download in video or audio. A complete list of the settings is below). The interface is used to configure the settings. Run the interface cell to open the interface. Once you've configured the parameters, press the `Download` button. A progress bar will appear.

### URL

Change the `URL` field to be that of the desired video or playlist.

- If you just want to see how AlteraAudio works, but don't particularly care what video it downloads, you can type `example` into the `URL` field to download an example video. This command will never give you up nor let you down.

#### Playlists

When given a YouTube playlist, AlteraAudio will iterate through each video in the list, taking the position in the playlist as the track number and the name of the playlist as the album title.

 - Example Playlist: `https://www.youtube.com/playlist?list=PLpR23XfB8-1WiOCUUDBNmBvCYy0jEMLEb`


### Format

The `Format` dropdown has two options:

- `mp3` is an audio format that is ubiquitous but lossy
- `m4a` is an audio format that is lossless but isn't universally recognized
- `mp4` is a video format rather than an audio file. No metadata will be attached to video files

### Download

The `Download` checkmark allows you to decide whether or not to automatically download the output to your device. Either way, the file gets saved in the Colab runtime (accessible from the file explorer on the left panel), so if you uncheck it by accident you can still initialize a download from the file explorer.

### Metadata

The `Metadata` checkmark determines whether or not AlteraAudio will attach metadata to audio files. This setting negates all the options below it, and it itself can be negated by the `Format` option (because we don't attach metadata to videos).

### Genre

YouTube doesn't categorize songs by genre, so the `Genre` field allows you to specify the genre.

### Album Artist

Similar to the `Genre` field, the `Album Artist` field allows you to specify the album artist, which isn't accesible from YouTube's database.

### Track Numbers

The `Track Numbers` checkmark allows you to specify whether or not AlteraAudio will set the track number of the audio file. There isn't a big use case for this setting, but if you need it it's availible.

## Troubleshooting

### Can I use AlteraAudio on a mobile device?

Yes? I've used AlteraAudio on my phone a few times, and it *works*, just not *well*. I'd highly suggest using a desktop client, but if you simply must use a mobile device, I'd suggest using it in landscape mode, as Colab crops the text otherwise.

### How do I run a cell?

Press the little arrow button at the top left corner.

### It says I need to sign in

You need a Google account to use Colab. If you don't have one, make one, and if you do have one, use it to sign in.

### It's not letting me connect to a runtime

In order to run scripts (like AlteraAudio), Colab connects to one of Google's computers and makes *it* do the processing. If there isn't a computer available, Colab will say it isn't able to connect. There isn't an easy way to fix this, so you'll just have to wait for a spot to open up.

### How do I know what the URL of my video is?

Go to YouTube, navigate to your video, and select 'Share'. Then select 'Copy Link'. Back in AlteraAudio, paste the link into the 'URL' field.

If you can't access the YouTube website, I made another notebook to get around that: [SatelliteSearch](https://colab.research.google.com/drive/1WjWwWbYvqxCabLJNQuD2ldPG2HqrlgWw#scrollTo=a3BQttF0xcsM).

### I'm not seeing the cover art

If you can't see the cover art after downloading the file, it could be an error with AlteraAudio, but it's far more likely to be your media player. Windows Media Player in particular has a penchant for not reading metadata. There isn't really a way to fix it other than installing a [better media player](https://getmusicbee.com/). (Note, if you've figured out a way to fix this, please [get in touch](mailto:29sr494d5@mozmail.com?subject=AlteraAudio%20Support) so I can implement it)

### The cover art is different from the album art

AlteraAudio can't get YouTube Music album covers. The code library I use doesn't have the capability. In most cases, the cover art of each song is the same as the album cover, although plenty of YouTube channels ignore this guideline. There's no way to fix this other than manually editing the cover art. To do this, see the below section about metadata editing.

### Changing the Metadata

AlteraAudio isn't a metadata editing software. If you want to (for example) change the album art to something else, you can do so with a dedicated metadata editor.

 - Many modern media players (including Windows Media Player) come included with a metadata editor, so you could always just use those.
 - You could also get a third party editor, such as an [online website](https://www.mp3tag.de/en/) or a [dedicated application](https://www.microsoft.com/store/productId/9NN77TCQ1NC8?ocid=pdpshare)
 - If there's a demand for it, I might make a metadata editor in Colab. I don't plan on it (I usually make things primarily for my own personal use, and I'm content with the metadata editor in [the media player I use](https://getmusicbee.com/)), but if that's something you'd want me to make, feel free to [get in touch](mailto:29sr494d5@mozmail.com?subject=AlteraAudio%20Support).

### Slow Downloads

There are a few reasons that could cause slow downloads. Here are a few:

 - Large video size. The longer the video is in length, the longer AlteraAudio will take to download it. To see if this is the problem, try again with the example song. The example song usually takes about 12 seconds to process in audio mode. As long as it takes less than, say, 20 seconds, the size of the selected video probably isn't causing the problem.
 - Slow internet speed. AlteraAudio uses the internet to download files from YouTube's servers to your device. To increase connection speeds, move your device closer to your router or connect to a different network.
 - Google server overload. It's possible that an unusal number of people are using YouTube or Colab, meaning that Google's servers will take longer to process your request. This scenario is unlikely, but there's really nothing you can do if it happens other than to try again later.

### Resolution

AlteraAudio is limited to 720p. The library AlteraAudio uses, PyTube, is limited to 720p, so if you want to download videos at a higher resolution, I made another notebook called [DeltaDownload](https://colab.research.google.com/drive/1J0ISLEfYjbUVZrSs_1oTYuvi4HzYdzzc) that uses a different library.

### Error Message

If AlteraAudio ever outputs an error, it means something has gone wrong. Here are some things you can try.

 - Make sure you've run the `Setup` block. Without it, AlteraAudio *cannot run*.
 - Try again with a different video. It's possible that AlteraAudio can't process that video, or the URL is invalid. To see if this is the case, see if it still works with a different video.
 - Restart the Colab runtime. Common advice for any computer problem is to turn it off and back on again. The equivalent in Colab is going to `Runtime` at the top and selecting 'Disconnect and delete runtime'. This will restart everything, so you'll need to rerun the `Setup` block.
 - If you've tried all of these and it still doesn't work, YouTube probably changed something, breaking PyTube. Please [email me](mailto:29sr494d5@mozmail.com?subject=AlteraAudio%20Support) with your error so I can fix it.

## Interface

In [ ]:
# @title IPyWidgets Interface { vertical-output: true }
# @markdown This cell creates the interface and allows you to specify operation parameters. You don't need to read any of this code (but you can if you want), just run the cell and wait 40-ish the first time (almost instant subsequently) seconds for the inputs to pop up.

# This line creates the title.
title_label = Label('AlteraAudio').add_class("para")

# This line creates a text input field where user can enter a URL.
# The default value is an empty string and the description displayed next to the field is "URL:".
url_input = Text(value="", description="URL:", layout = Layout(width='100ch'))

# This line creates a dropdown menu with two options: "Audio" and "Video".
# The default selected value is "Audio" and the description displayed next to the menu is "Format".
format_dropdown = Dropdown(
    options=["mp3", "m4a", "mp4"],
    value="m4a",
    description="Format"
)

# This line creates a collapsible section with a solid border and some padding.
# This section will house the "Advanced options".
advanced_box = VBox(children=[], layout=Layout(border="solid 1px", padding="5px"), description="Advanced")

# This line creares a label so the user understands that the advanced settings are optional.
advanced_label = Label('Advanced').add_class("para")

# This line creates a checkbox with the description "Download File" and sets the default state to checked (True).
download_checkbox = Checkbox(description="Download File", value=True)

# This line creates a checkbox with the description "Apply Metadata" and sets the default state to checked (True).
metadata_checkbox = Checkbox(description="Apply Metadata", value=True)

# This line creates a text input field where users can enter the genre of the media.
# The default value is an empty string and the description displayed next to the field is "Genre:".
genre_input = Text(description="Genre:", value="")

# This line creates a text input field where users can enter the album artist of the media.
# The default value is an empty string and the description displayed next to the field is "Album Artist:".
album_artist_input = Text(description="Album Artist:", value="")

# This line creates a checkbox with the description "Include Track Numbers" and sets the default state to checked (True).
track_numbers_checkbox = Checkbox(description="Include Track Numbers (only affects playlists)", value=True)

# This line adds all the previously created checkbox and text input elements as children to the "advanced_box" section.
advanced_box.children = [
    download_checkbox,
    metadata_checkbox,
    genre_input,
    album_artist_input,
    track_numbers_checkbox
]

# This line displays the "url_input", "format_dropdown" and "advanced_box" elements on the screen.
display(title_label, url_input, format_dropdown, advanced_label, advanced_box)

# Download button function

def download_button_clicked(b):
  try:
    # Print an empty line for formatting
    print()

    # Get the selected format (either "Audio" or "Video") from the dropdown menu and convert it to lowercase.
    format = format_dropdown.value.lower()

    extension = format

    filepath = altera(
        url_input.value,
        extension,
        genre=genre_input.value,
        albArtist=album_artist_input.value,
        sequential=track_numbers_checkbox.value,
        apply_metadata=metadata_checkbox.value
    )

    # Download the file only if the "download_checkbox" is checked and the environment is "colab".
    if download_checkbox.value and environment == "colab":
        # Download the file using the "files.download" function (likely from a specific library)
        files.download(filepath)
  except ImportError:
    print("You did not run the Setup block. Read the documentation and try again.")

download_button = Button(description="Download")
download_button.on_click(download_button_clicked)

display(download_button)